In [3]:
!pip install --quiet numpy pandas matplotlib ISLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.0 MB/s eta 0:00:00


In [10]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

In [11]:
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [12]:
auto = load_data('Auto')
a_train, a_valid = train_test_split(auto,
                                    test_size=196,
                                    random_state=0)
a_train.shape, a_valid.shape

((196, 8), (196, 8))

In [14]:
hp_ms = MS(['horsepower'])

X_train = hp_ms.fit_transform(a_train)
y_train = a_train['mpg']

model = sm.OLS(y_train, X_train)
result = model.fit()


summarize(result)

,coef,std err,t,P>|t|
intercept,39.9055,1.009,39.537,0.0
horsepower,-0.1563,0.009,-17.333,0.0


In [15]:
x_valid = hp_ms.transform(a_valid)
y_valid = a_valid['mpg']

valid_pred = result.predict(x_valid)
np.mean((y_valid - valid_pred) ** 2)

23.61661706966988

In [16]:
# creating a validation set and evaluating using MSE.
def eval_MSE(terms, response,
             train, test):
    mm = MS(terms)

    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean( (y_test - test_pred) ** 2)

In [17]:
mse = np.zeros(3)

for idx, degree in enumerate(range(1, 4)):
    mse[idx] = eval_MSE([poly('horsepower', degree)],
                        'mpg',
                        a_train, a_valid)

mse

array([23.61661707, 18.76303135, 18.79694163])

In [18]:
# For cross-validation

hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))

X, Y = auto.drop(columns = ['mpg']), auto['mpg']

cv_results = cross_validate(hp_model, X, Y, cv = auto.shape[0])
cv_err = np.mean(cv_results['test_score'])

cv_err

24.23151351792922

In [20]:
cv_error = np.zeros(5)

hp = np.array(auto['horsepower'])
mm = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1, 6)):
    X = np.power.outer(hp, np.arange(d+1))
    m_cv = cross_validate(mm, X, Y, cv = auto.shape[0])

    cv_error[i] = np.mean(m_cv['test_score'])

cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443029, 19.03320648])